## Assignment 1, Part 1 – Transformations for Normalized Power


In [ ]:
## Assignment 1, Part 1 – Transformations for Normalized Power
## Based on Lecture 4 (Example 4.16) and assignment equations (1) & (2)

# ---------------------------------------------------------------
# 1. Load and normalize data
# ---------------------------------------------------------------
df_tuno <- read.table("data/tuno.txt", header = TRUE)
df_tuno$pow.obs <- df_tuno$pow.obs / 5000        # normalize to (0,1)
y <- df_tuno$pow.obs
summary(df_tuno)
head(df_tuno)


### Defining Box-cox transformations

In [ ]:
y <- df_tuno$pow.obs   # original data (positive, scaled if needed)
n <- length(y)

# --- Standard Box–Cox (Lecture 4)
bc.trans <- function(lambda, y) {
  if (lambda == 0) log(y) else (y^lambda - 1) / lambda
}

# --- Assignment transformation (1)
# y(λ) = (1/λ) * log( y^λ / (1 - y^λ) )
trans1 <- function(lambda, y) {
  (1 / lambda) * log((y^lambda) / (1 - y^lambda))
}

# --- Assignment transformation (2)
# y(λ) = 2 * log( y^λ / (1 - y)^(1 - λ) )
trans2 <- function(lambda, y) {
  2 * log((y^lambda) / ((1 - y)^(1 - lambda)))
}


### Profile likelihood

In [ ]:
# ---------------------------------------------------------------
# 3. Profile log-likelihood function (from Lecture 4)
# ---------------------------------------------------------------
lp.lambda <- function(lambda, y, trans.fun) {
  y.l <- trans.fun(lambda, y)
  if (any(!is.finite(y.l))) return(NA)     # skip invalid λ
  n <- length(y)
  sigmasq <- mean((y.l - mean(y.l))^2)
  -n/2 * log(sigmasq)  + (lambda - 1) * sum(log(y))
}

In [ ]:
# ---------------------------------------------------------------
# 4. Compute profile likelihoods
# ---------------------------------------------------------------
lambda.seq.bc <- seq(0.2, 0.5, by = 0.01)
lambda.seq.t1 <- seq(0.2, 0.5, by = 0.01)
lambda.seq.t2 <- seq(0.001, 0.2, by = 0.01)

lp.bc <- sapply(lambda.seq.bc, lp.lambda, y = y, trans.fun = bc.trans)
lp.t1 <- sapply(lambda.seq.t1, lp.lambda, y = y, trans.fun = boxcox)
lp.t2 <- sapply(lambda.seq.t2, lp.lambda, y = y, trans.fun = trans2)

# ---------------------------------------------------------------
# 5. Plot profile likelihoods
# ---------------------------------------------------------------
par(mfrow = c(1,3), bg = "white")
plot(lambda.seq.bc, lp.bc - max(lp.bc, na.rm=TRUE), type = "l",
     main = "Profile Likelihood (Box–Cox)",
     xlab = expression(lambda), ylab = "Relative log-likelihood")
abline(h = -qchisq(0.95, df = 1)/2, col = 2, lty = 2)

plot(lambda.seq.t1, lp.t1 - max(lp.t1, na.rm=TRUE), type = "l",
     main = "Profile Likelihood (Eq. 1)",
     xlab = expression(lambda), ylab = "Relative log-likelihood")
abline(h = -qchisq(0.95, df = 1)/2, col = 2, lty = 2)

plot(lambda.seq.t2, lp.t2 - max(lp.t2, na.rm=TRUE), type = "l",
     main = "Profile Likelihood (Eq. 2)",
     xlab = expression(lambda), ylab = "Relative log-likelihood")
abline(h = -qchisq(0.95, df = 1)/2, col = 2, lty = 2)
par(mfrow = c(1,1))

In [ ]:
# ---------------------------------------------------------------
# 6. Find optimal λ for each transformation (Lecture 4 syntax)
# ---------------------------------------------------------------
opt.bc <- optimize(lp.lambda, c(-2, 2), y = y, trans.fun = bc.trans, maximum = TRUE)$maximum
opt.t1 <- optimize(lp.lambda, c(0.001, 2), y = y, trans.fun = boxcox, maximum = TRUE)$maximum
opt.t2 <- optimize(lp.lambda, c(0.001, 0.999), y = y, trans.fun = trans2, maximum = TRUE)$maximum

opt.bc; opt.t1; opt.t2

In [ ]:
# ---------------------------------------------------------------
# 8. Fit distributions and compare (choose best transformation)
# ---------------------------------------------------------------
y.trans <- y.t1   # Example: Eq. (1) works well again
y.raw   <- y

# --- Normal fit
fit.normal <- fitdistr(y.trans, densfun = "normal")
normal.mean <- fit.normal$estimate["mean"]
normal.sd   <- fit.normal$estimate["sd"]
normal.mean

In [ ]:
# ---------------------------------------------------------------
# 9. MLEs for mu and sigma (normal model)
# ---------------------------------------------------------------
mu.hat    <- normal.mean
sigma.hat <- normal.sd
n         <- length(y.trans)

mu.hat
sigma.hat


In [ ]:
# ---------------------------------------------------------------
# 10. Fisher information and standard errors
# ---------------------------------------------------------------
# Standard error for mu
se.mu <- sigma.hat / sqrt(n)

# Standard error for sigma (delta method)
se.sigma <- sigma.hat / sqrt(2 * n)

se.mu
se.sigma


In [ ]:
# ---------------------------------------------------------------
# 11. Wald confidence intervals
# ---------------------------------------------------------------
alpha <- 0.05
z     <- qnorm(1 - alpha/2)

ci.mu <- c(
  mu.hat - z * se.mu,
  mu.hat + z * se.mu
)

ci.sigma <- c(
  sigma.hat - z * se.sigma,
  sigma.hat + z * se.sigma
)

ci.mu
ci.sigma
